Alright — we’ll keep running this as a repeating CPSE-style interview loop.

## Phase 1: Coding (Difficulty ~3/10)

### Problem: Detect Error-Rate Spikes per Tenant (1-minute windows)

Implement:

```python
def detect_error_spikes(logs: list[str], error_rate_threshold: float, min_requests: int) -> list[tuple[str, int]]:
    ...
```

Each log line is supposed to be:

`"<timestamp_sec>,<tenant_id>,<status_code>"`

Example:

* `"1700000123,tenantA,500"`
* `"1700000155,tenantA,200"`
* `"1700000188,tenantB,503"`

### Definitions

* A **1-minute window** is based on `timestamp_sec`:

  * `window_start = (timestamp_sec // 60) * 60`
* A request is an **error** if `500 <= status_code <= 599`
* For each `(tenant_id, window_start)`, compute:

  * `total_requests`
  * `error_requests`
  * `error_rate = error_requests / total_requests`

### Output

Return a list of `(tenant_id, window_start)` for every window where:

* `total_requests >= min_requests`, AND
* `error_rate >= error_rate_threshold`

Sort output by:

1. `window_start` ascending
2. `tenant_id` alphabetical

### Data Quality Rules

* Ignore malformed lines silently (don’t raise)
* Ignore lines where timestamp/status code are not integers
* Ignore empty lines or lines starting with `#`

### Example

If in window `1700000100` tenantA has 10 total requests and 3 are 5xx, error_rate = 0.3.
If `error_rate_threshold = 0.25` and `min_requests = 5` → tenantA triggers for that window.

---

### What I want from you

1. Your approach (short)
2. Clean Python code
3. Minimal tests
4. Big-O time + space

When you’re done, I’ll give you a **coding rating (1–10)** with brief improvement areas, then we’ll pivot to **Phase 2: “How would this work for 100 million users?”** (high-level CPSE system thinking + troubleshooting).

You can ask clarification questions first if you want.
---
---
Here’s a concrete sample you can use.

### Sample Input

```python
logs = [
    "# comment line",
    "1700000101,tenantA,200",
    "1700000102,tenantA,500",
    "1700000103,tenantA,503",
    "1700000104,tenantA,200",

    "1700000120,tenantB,500",
    "1700000121,tenantB,200",
    "1700000122,tenantB,200",
    "1700000123,tenantB,200",

    "1700000165,tenantA,500",
    "1700000166,tenantA,500",
    "1700000167,tenantA,200",
    "bad_line",
    "1700000180,tenantA,abc",
]

error_rate_threshold = 0.50
min_requests = 4
```

### How windows break down

* Window `1700000100` (timestamps 1700000100–1700000159)

  * tenantA: total 4, errors 2 → error_rate = 0.50 ✅ (meets threshold, total >= 4)
  * tenantB: total 4, errors 1 → error_rate = 0.25 ❌
* Window `1700000160` (timestamps 1700000160–1700000219)

  * tenantA: total 3 (valid), errors 2 → total < 4 ❌

### Sample Output

```python
[("tenantA", 1700000100)]
```
---
---
---

In [ ]:
def detect_error_spikes(logs: list[str], error_rate_threshold: float, min_requests: int) -> list[tuple[str, int]]:
    counts = {}   # {(tenant_id, window_start): (total_requests, error_requests)}
    result = []

    # Pass 1: count requests per tenant per time window
    for log in logs:
        log = log.strip()
        ts_str, tenant_id, code_str = log.split(",")

        ts = int(ts_str)
        error_code = int(code_str)

        ws = (ts // 60) * 60
        key = (tenant_id, ws)

        if key not in counts:
            counts[key] = (0, 0)

        totals, errors = counts[key]
        totals += 1

        if 500 <= error_code <= 599:
            errors += 1

        counts[key] = (totals, errors)

    # Pass 2: evaluate each window once
    for key, (totals, errors) in counts.items():
        if totals >= min_requests:
            error_rate = errors / totals
            if error_rate >= error_rate_threshold:
                result.append(key)

    return result


print(detect_error_spikes([
    "1700000101,tenantA,200",
    "1700000102,tenantA,500",
    "1700000103,tenantA,503",
    "1700000104,tenantA,200",
    "1700000120,tenantB,500",
    "1700000121,tenantB,200",
    "1700000122,tenantB,200",
    "1700000123,tenantB,200",
    "1700000165,tenantA,500",
    "1700000166,tenantA,500",
    "1700000167,tenantA,200"
], 0.5, 4))

[]


Absolutely — this kind of problem is hard at first because it asks you to do **3 things at once**:

1. **Parse input**
2. **Group data**
3. **Check a condition on each group**

That is a very common pattern in coding interviews.

---

## 1) Your code with plain-English comments

```python
def detect_error_spikes(logs: list[str], error_rate_threshold: float, min_requests: int) -> list[tuple[str, int]]:
    # This dictionary will store counts for each (tenant, time_window).
    # Key:   (tenant_id, window_start)
    # Value: (total_requests_in_that_window, error_requests_in_that_window)
    counts = {}

    # This will store the final answers that match the rule.
    result = []

    # First pass:
    # Go through every log line and build our grouped counts.
    for log in logs:
        # Remove extra spaces/newlines
        log = log.strip()

        # Split the log into 3 parts:
        # timestamp, tenant, status_code
        ts_str, tenant_id, code_str = log.split(",")

        # Convert timestamp and status code from string to integer
        ts = int(ts_str)
        error_code = int(code_str)

        # Find which 60-second window this timestamp belongs to.
        # Example:
        # 1700000101 -> 1700000100
        # 1700000123 -> 1700000100
        ws = (ts // 60) * 60

        # Make a grouping key using tenant + time window
        key = (tenant_id, ws)

        # If this tenant/window has not been seen before,
        # start its counters at (0 total requests, 0 errors)
        if key not in counts:
            counts[key] = (0, 0)

        # Read the current counters for this tenant/window
        totals, errors = counts[key]

        # Every log line is one request, so increase total count
        totals += 1

        # If the status code is a server error (500 to 599),
        # increase the error count
        if 500 <= error_code <= 599:
            errors += 1

        # Save the updated counts back into the dictionary
        counts[key] = (totals, errors)

    # Second pass:
    # Now check each tenant/window group once.
    for key, (totals, errors) in counts.items():
        # Ignore small groups that do not have enough requests
        if totals >= min_requests:
            # Compute the error rate for this group
            error_rate = errors / totals

            # If the error rate is high enough, add this key to the result
            if error_rate >= error_rate_threshold:
                result.append(key)

    # Return all matching (tenant_id, window_start) pairs
    return result
```

---

## Mental model for this code

Think of it like this:

### Step A: Read one log

Example:

```python
"1700000102,tenantA,500"
```

Extract:

* timestamp = `1700000102`
* tenant = `tenantA`
* code = `500`

### Step B: Decide its bucket

Put that request into a bucket:

* tenant = `tenantA`
* minute window = `1700000100`

So bucket key becomes:

```python
('tenantA', 1700000100)
```

### Step C: Update that bucket

For that bucket, track:

* total requests
* error requests

So maybe:

```python
('tenantA', 1700000100) -> (4 total, 2 errors)
```

### Step D: After all logs, check each bucket

Now ask:

* Does it have at least `min_requests`?
* Is `errors / total >= threshold`?

If yes, include it.

---

## 2) Tips for similar problems

### Tip 1: Split the problem into **“build”** and **“check”**

A lot of beginner struggle comes from trying to do everything in one loop.

Instead, think:

* **Pass 1:** Build grouped data
* **Pass 2:** Check conditions

That alone makes many problems much easier.

---

### Tip 2: Ask: “What is my grouping key?”

In this problem, the key is:

```python
(tenant_id, time_window)
```

In other problems, the key might be:

* `(user_id, day)`
* `(product_id, month)`
* `(student_name, subject)`

When a problem says “for each X per Y”, it usually means:
**use a dictionary with a grouped key**.

---

### Tip 3: Decide what value you need to store per key

Once you know the key, ask:

**What information must I track for each group?**

Here, for each `(tenant, window)`, you need:

* total requests
* error requests

That becomes the dictionary value.

---

### Tip 4: Make one fake example by hand

Before coding, write one or two inputs and manually track them:

* What is the key?
* What changes in the dictionary?

This helps your brain see the structure before writing syntax.

---

### Tip 5: Watch for “duplicate result” bugs

If you append results while still building counts, you may add the same group many times.

A safe default:

* build first
* append answers after

---

## 3) Very similar practice problem

Try this on your own:

### Problem: Detect slow response spikes

You are given API logs in this format:

```python
"1700000101,tenantA,120"
"1700000102,tenantA,900"
"1700000103,tenantA,850"
```

Each line means:

* timestamp
* tenant_id
* response_time_ms

Write:

```python
def detect_slow_windows(logs: list[str], slow_ms: int, min_requests: int) -> list[tuple[str, int]]:
```

Return all `(tenant_id, window_start)` where:

* the window is still a 60-second bucket
* the tenant has at least `min_requests` requests in that window
* **at least half** of the requests in that window have response time `>= slow_ms`

---

### Example

Input:

```python
logs = [
    "1700000101,tenantA,100",
    "1700000102,tenantA,900",
    "1700000103,tenantA,850",
    "1700000104,tenantA,200",

    "1700000120,tenantB,700",
    "1700000121,tenantB,100",
    "1700000122,tenantB,100",
    "1700000123,tenantB,100"
]
slow_ms = 800
min_requests = 4
```

Expected output:

```python
[("tenantA", 1700000100)]
```

Because:

* `tenantA` in window `1700000100` has 4 requests
* 2 of them are slow (`900`, `850`)
* `2 / 4 = 0.5`, so it qualifies

---

## How you should think before solving that practice problem

Use the exact same structure:

* key = `(tenant_id, window_start)`
* value = `(total_requests, slow_requests)`

Then:

1. Build counts
2. Check each group

---

If you want, next I can give you:

* a **pseudo-code version** of the practice problem first, or
* let you try it yourself and I’ll review your answer like an interviewer.
